In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


## 1) La capitalisation boursière en tant que taille moyenne de l'entreprise x nombre d'entreprises

In [3]:
market_cap = df_numfirm.multiply(df_avgsize, axis=0)

## 2) Le rapport entre la valeur comptable (BE : book equity) et la valeur de marché (ME : market equity), c'est-à-dire le rapport book-to-market, en utilisant les données "Sum of BE / Sum of ME".

### (Remarque : les données "Somme de BE / Somme de ME" sont annuelles et doivent être converties en données mensuelles en supposant que le ratio reste constant entre juillet de l'année s et juin de l'année s + 1. Par exemple, le ratio de 1926 est le ratio qui doit être utilisé pour la période allant de juillet 1926 à juin 1927).

In [4]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_BtoM = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

df_BtoM['Date'] = pd.to_datetime(df_BtoM['Date'], format='%Y')

df_BtoM = df_BtoM.drop('Date', axis=1)

df_BtoM = pd.DataFrame(data = df_BtoM.iloc[5:].values, index = df_48ind.index, columns = df_48ind.columns)

df_BtoM.replace(-99.99, np.nan, inplace = True)
df_BtoM.replace(-999, np.nan, inplace = True)
df_BtoM.dropna(inplace = True)

## 3) Le momentum de chaque industrie est le rendement moyen de cette industrie au cours des 12 derniers mois, y compris le mois t.

In [5]:
# Calculer la moyenne mobile sur 12 mois pour chaque industrie
df_mom = df_48ind.rolling(window=12).mean()

df_mom.replace(-99.99, np.nan, inplace = True)
df_mom.replace(-999, np.nan, inplace = True)
df_mom.dropna(inplace = True)

## Pour chaque mois t, standardiser chaque caractéristique de manière transversale pour avoir une moyenne nulle et un écart-type unitaire pour toutes les actions à la date t, comme expliqué dans Brandt et al. (2009).

In [6]:
# Créer un objet StandardScaler
scaler = StandardScaler()

def standardize(df):
    # Standardiser les données
    df_scaled = scaler.fit_transform(df)
    
    df_standardized = pd.DataFrame(df_scaled, columns=df.columns, index=df.index)
    
    return df_standardized

standardized_market_caps = standardize(market_cap)
standardized_BtoM = standardize(df_BtoM)
standardized_mom = standardize(df_mom)

market_weights = market_cap.div(market_cap.sum(axis=1), axis=0)

In [20]:
optimal_weights = market_weights + (1 / len(market_weights.columns)) * (optimal_beta[0] * standardized_market_caps + optimal_beta[1] * standardized_BtoM + optimal_beta[2] * standardized_mom)

optimal_weights.dropna(inplace = True)
optimal_weights

# Code original

In [30]:
import numpy as np
from scipy.optimize import minimize

def u(x, gamma):
    return (x ** (1 - gamma) - 1) / (1 - gamma)

# Fonction à maximiser
def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
    wi_t = np.random.rand(N) # Supposons que vous avez une fonction pour calculer wi_t
    return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)), gamma) * (1 + r))

# Données
T = 54  # Nombre d'observations
N = 48   # Nombre d'actifs
MC = np.random.rand(T, N)  # Exemple de données de market cap
BM = np.random.rand(T, N)  # Exemple de données de book-to-market ratio
MOM = np.random.rand(T, N) # Exemple de données de momentum
r = np.random.rand(T)       # Exemple de données de rendement
gamma = 0.5  # Paramètre de la fonction d'utilité CRRA

# Initialisation des coefficients beta
initial_beta = np.array([0.5, 0.5, 0.5])

# Optimisation des coefficients beta
result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

# Résultats
optimal_beta = result.x
maximized_value = -result.fun

print("Coefficients β optimaux :", optimal_beta)
print("Valeur maximisée de la fonction :", maximized_value)


Coefficients β optimaux : [671194.56469017 380780.8218448  690714.58475625]
Valeur maximisée de la fonction : 20791.298805983857


/var/folders/ns/nlrkcsjd7j70p9jvf2jl5t4c0000gn/T/ipykernel_3582/1715546139.py:5: RuntimeWarning: invalid value encountered in scalar power
  return (x ** (1 - gamma) - 1) / (1 - gamma)


# ******

In [33]:
def maximize_utility(MC, BM, MOM, r, T, N, gamma, start, end):
    
    def u(x, gamma):
        return x ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma):
        wi_t = market_weights.loc[start:end]
        return -(1/T) * np.sum(u(np.sum(wi_t + (1/N) * (beta[0] * MC.loc[start:end] + beta[1] * BM.loc[start:end] + beta[2] * MOM.loc[start:end])), gamma) * (1 + r))

    # Initialisation des coefficients beta
    initial_beta = np.array([0.5, 0.5, 0.5])

    # Optimisation des coefficients beta
    result = minimize(function_to_maximize, initial_beta, args=(MC, BM, MOM, r, T, N, gamma), method='SLSQP')

    # Résultats
    optimal_beta = result.x

    return optimal_beta

start_date = pd.to_datetime('1969-07-01')
end_date = pd.to_datetime('1973-12-01')

maximize_utility(standardized_market_caps, standardized_BtoM, standardized_mom, np.random.rand(48), 48, 48, 0.5, start_date, end_date)

array([-7.18817049e+10,  5.17125394e+10, -6.89966157e+10])

In [34]:
import numpy as np
from scipy.optimize import minimize

def maximize_utility(MC, BM, MOM, r, T, N, gamma):

    def u(x, gamma):
        return x ** (1 - gamma) / (1 - gamma)

    def function_to_maximize(beta, MC, BM, MOM, r, T, N, gamma, wi_t):
        return -(1/T) * u(np.sum(wi_t + (1/N) * (beta[0] * MC + beta[1] * BM + beta[2] * MOM)), gamma) * (1 + r)

    # Initialisation des coefficients beta
    initial_beta = np.array([0.5, 0.5, 0.5])

    optimal_betas = []

    for i in range(len(MC)):
        wi_t = market_weights.iloc[i]
        result = minimize(function_to_maximize, initial_beta, args=(MC.iloc[i], BM.iloc[i], MOM.iloc[i], r, T, N, gamma, wi_t), method='SLSQP')
        optimal_betas.append(result.x)

    return optimal_betas

start_date = pd.to_datetime('1969-07-01')
end_date = pd.to_datetime('1973-12-01')


market_weights
MC = standardized_market_caps.loc[start_date:end_date]
MC = standardized_market_caps.loc[start_date:end_date]
MC = standardized_market_caps.loc[start_date:end_date]


maximize_utility(MC, BM, MOM, r, T, N, gamma)

In [ ]:
optimal_weights = market_weights + (1 / len(market_weights.columns)) * (optimal_beta[0] * standardized_market_caps + optimal_beta[1] * standardized_BtoM + optimal_beta[2] * standardized_mom)

optimal_weights.dropna(inplace = True)
optimal_weights